# Gensim-Doc2vec Training

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('preprocessed_news_218posts.csv', sep='|')

In [3]:
df.head(1)

,item_id,date,title,content,sentiment,summary,top_key_freq,tokens,tokens_rm_stopwords,entities,token_pos,link,photo_link,category
0,aipl_20220929_1,2022-09-29,台灣國際漁業展、智慧農業週開幕 逾15國參展,外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式，他表示...,0.68,"['外交部政次田中光今天出席2022年「台灣國際漁業展」與「台灣智慧農業週」聯合開幕儀式',...","[('展覽', 8), ('台灣', 6), ('廠商', 6), ('參展', 6), (...","['外交部', '政次', '田中光', '今天', '出席', '2022年', '「',...","['外交部', '政次', '田中光', '出席', '台灣', '國際', '漁業展', ...","{(657, 658, 'GPE', '台'), (143, 146, 'PERSON', ...","[('外交部', 'Nc'), ('政次', 'Na'), ('田中光', 'Nb'), (...",https://www.cna.com.tw/news/aipl/202209290375....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,政治


# Prepare training data with TaggedDocument format

In [4]:
import gensim

In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

## tokens_rm_stopwords and item_id are used to train the word2vec model 

Using tokens_rm_stopwords is better than tokens, because tokens_rm_stopwords contains only import keywords. The stop words and some unnecessary POS were removed.

In [6]:
df.tokens_rm_stopwords[0]

"['外交部', '政次', '田中光', '出席', '台灣', '國際', '漁業展', '台灣', '智慧', '農業', '儀式', '廠商', '參展', '展覽', '規模', '參展', '國家', '國內外', '廠商', '使節', '出席', '展覽', '馬紹爾', '群島', '大使', '艾芮瓊', '聖文森', '格瑞那丁', '大使', '柏安卓', '德經', '辦事處長', '林百科', '馬來西亞', '友誼', '貿易', '中心', '代表', '何瑞萍', '台北', '印尼', '經濟', '貿易', '代表', '貿易', '主任', '蘇迪佑', '匈牙利', '貿易', '辦事處', '商務', '投資', '主任', '戴博斯', '田中光', '台灣', '智慧', '台灣', '國際', '漁業展', '台灣', '漁產', '業界', '國際性', '嘉年華', '展覽', '展覽', '規模', '參展', '國家', '國內外', '廠商', '參展', '廠商', '產學', '機構', '台灣', '漁業', '優質', '產品', '技術', '國際', '展覽', '訂閱', '世界', '電子報', '掌握', '天下', '輸入', '電子', '信箱', '格式', '訂閱', '訂閱', '田中光', '國家', '廠商', '參展', '吐瓦魯國', '大使館', '巴布亞', '紐幾內亞', '商務', '代表', '聖文森', '格瑞那丁', '大使館', '史瓦帝尼', '王國', '大使館', '匈牙利', '貿易', '辦事處', '參展', '攤位', '國際', '廠商', '外交團', '展覽', '國際性', '異國', '風采', '田中光', '世界', '經濟', 'COVID-19', '疫情', '衝擊', '實體', '展覽', '台灣版', '漁業', '經貿', '泡泡', '經營', '展覽', '規模', '田中光韓', '代表', '半導體業', '獲益', '印尼', '舉辦', '酒會', '代表', '經貿', '關係']"

In [7]:
df.tokens_rm_stopwords

0      ['外交部', '政次', '田中光', '出席', '台灣', '國際', '漁業展', ...
1      ['兩岸', '局勢', '美國', '國務卿', '蓬佩奧', '接受', '媒體', '...
2      ['民進黨', '新北市長', '參選人', '林佳龍', '立委', '余天合體', '造...
3      ['韓國', '代表', '鄭炳元', '韓國', '酒會', '致詞', '產業', '互...
4      ['中央社', '記者', '郭建伸', '王承中', '台北', '台南', '殺警案',...
                             ...                        
213    ['香港', '立法會', '議員', '許智峯', '保釋', '期間', '逃亡', '...
214    ['亞洲', '放寬', '邊境', '中國', '推動', '清零', '方針', '山西...
215    ['出席', '日相', '安倍', '國葬', '中國', '政協', '副主席', '萬...
216    ['中國', '本土', 'COVID-19', '病例', '北京', '陽性', '人員...
217    ['中國', '大陸', '國台辦', '批評', '美國', '國務卿', '蓬佩奧', ...
Name: tokens_rm_stopwords, Length: 218, dtype: object

In [8]:
len(df)

218

## Prepare training data with TaggedDocument format

In [9]:
# Prepare the tagged document 
from gensim.models.doc2vec import TaggedDocument
documents = []
for i in range( len(df) ):
    tokens = df.iloc[i].tokens_rm_stopwords
    words = eval( tokens ) # convert the format of tokens from string to list
    item_id = df.iloc[i].item_id # item_id is with string format
    tag_id = [item_id] # the tag_id should be placed in a list
    documents.append(TaggedDocument(words, tag_id))

In [10]:
documents[0]

TaggedDocument(words=['外交部', '政次', '田中光', '出席', '台灣', '國際', '漁業展', '台灣', '智慧', '農業', '儀式', '廠商', '參展', '展覽', '規模', '參展', '國家', '國內外', '廠商', '使節', '出席', '展覽', '馬紹爾', '群島', '大使', '艾芮瓊', '聖文森', '格瑞那丁', '大使', '柏安卓', '德經', '辦事處長', '林百科', '馬來西亞', '友誼', '貿易', '中心', '代表', '何瑞萍', '台北', '印尼', '經濟', '貿易', '代表', '貿易', '主任', '蘇迪佑', '匈牙利', '貿易', '辦事處', '商務', '投資', '主任', '戴博斯', '田中光', '台灣', '智慧', '台灣', '國際', '漁業展', '台灣', '漁產', '業界', '國際性', '嘉年華', '展覽', '展覽', '規模', '參展', '國家', '國內外', '廠商', '參展', '廠商', '產學', '機構', '台灣', '漁業', '優質', '產品', '技術', '國際', '展覽', '訂閱', '世界', '電子報', '掌握', '天下', '輸入', '電子', '信箱', '格式', '訂閱', '訂閱', '田中光', '國家', '廠商', '參展', '吐瓦魯國', '大使館', '巴布亞', '紐幾內亞', '商務', '代表', '聖文森', '格瑞那丁', '大使館', '史瓦帝尼', '王國', '大使館', '匈牙利', '貿易', '辦事處', '參展', '攤位', '國際', '廠商', '外交團', '展覽', '國際性', '異國', '風采', '田中光', '世界', '經濟', 'COVID-19', '疫情', '衝擊', '實體', '展覽', '台灣版', '漁業', '經貿', '泡泡', '經營', '展覽', '規模', '田中光韓', '代表', '半導體業', '獲益', '印尼', '舉辦', '酒會', '代表', '經貿', '關係'], tags=['aipl_20220929_1'])

## Train the doc2vec model

In [11]:
from gensim.models.doc2vec import Doc2Vec

In [12]:
%%time
# Train model
# epochs = 5 
model = Doc2Vec(documents, vector_size = 100, window = 5, min_count = 1, workers = 4, epochs = 20 )

CPU times: total: 1.41 s
Wall time: 723 ms


# Save model

In [13]:
model.save("cna_news_doc2vec.model")

